In [ ]:
#Celem tego laboratorium było zapoznanie się z uczeniem maszynowym
#Pandy to podstawowe narzędzie, którego naukowcy używają do eksploracji i manipulowania danymi. Większość ludzi skraca pandy w swoim kodzie jako pd.
!pip install pandas

In [ ]:
### Zbieranie danych dotyczących cen domów w Melbourne
import pandas as pd
#Przykładowe dane (Melbourne) znajdują się w ścieżce pliku ../input/melbourne-housing-snapshot/melb_data.csv, a ich dodanie zostało przedstawione poniżej.
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
melbourne_data.columns

# Dropna pomija brakujące wartości z plików.
melbourne_data = melbourne_data.dropna(axis=0)
### Kolumny, które zostały wprowadzone do programu są traktowane jako funkcje i służą do prognozowania. W tym przypadku byłyby to kolumny do określenia ceny domu.
## Wybór wielu funkcji, podając listę nazw kolumn w nawiasach. Każda pozycja na tej liście powinna być ciągiem znaków (w cudzysłowach).
y = melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
## Te dane zostały nazwane X
X = melbourne_data[melbourne_features]
#Dane, których będziemy używać do przewidywania cen domów za pomocą metody opisywania i metody ceny podstawowej.
X.describe()
X.head()

### BUDOWANIE MODELU DRZEWA DECYZYJNEGO
###Do tworzenia modeli wykorzystano bibliotekę scikit-learning.
### Podczas kodowania ta biblioteka jest zapisywana jako sklearn. Scikit-learn jest z łatwością najpopularniejszą biblioteką do modelowania typów danych zwykle przechowywanych w DataFrames.

### Poniżej przedsawiłem kod budujący drzewo decyzyjne.
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
melbourne_model = DecisionTreeRegressor(random_state=1)

# Fit model
melbourne_model.fit(X, y)
##Wiele modeli uczenia maszynowego dopuszcza pewną przypadkowość w uczeniu modeli.
## Określenie liczby dla random_state zapewnia takie same wyniki w każdym przebiegu.

#Mamy teraz dopasowany model, którego możemy użyć do prognozowania.

#W praktyce trzeba będzie przewidywać nowe domy wchodzące na rynek, a nie domy, dla których mamy już ceny.
## Ale zrobimy prognozy dla pierwszych kilku wierszy danych uczących, aby zobaczyć, jak działa funkcja przewidywania.

print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(melbourne_model.predict(X.head()))



FileNotFoundError: ignored

In [ ]:
## WALIDACJA MODELU
#Co to jest walidacja modelu

## W większości (choć nie we wszystkich) zastosowaniach odpowiednią miarą jakości modelu jest dokładność predykcyjna. Innymi słowy, czy przewidywania modelu będą bliskie temu, co faktycznie się dzieje.
# Wiele osób popełnia ogromny błąd podczas pomiaru dokładności predykcyjnej. Tworzą prognozy na podstawie danych szkoleniowych i porównują te prognozy z wartościami docelowymi w danych szkoleniowych. Za chwilę zobaczysz problem z tym podejściem i jak go rozwiązać, ale zastanówmy się najpierw, jak to zrobimy.
#Najpierw trzeba podsumować jakość modelu w zrozumiały sposób. Jeśli zostaną porównane przewidywane i rzeczywiste wartości domów dla 10 000 domów, prawdopodobnie zostanie znalezione połączenie dobrych i złych prognoz. Przeglądanie listy 10 000 przewidywanych i rzeczywistych wartości byłoby bezcelowe. 
#Trzeba podsumować to w jednej metryce.
#Istnieje wiele metryk służących do podsumowania jakości modelu, ale zaczniemy od metryki o nazwie Średni błąd bezwzględny (zwanej również MAE). Dane zostaną podzielone, zaczynając od ostatniego słowa, błędu.

#Gdy mamy już model, oto jak obliczyć średni błąd bezwzględny:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

#Powyższą miarę można nazwać wynikiem „w próbce”. Została wykorzystana jedna „próbka” domów zarówno do zbudowania modelu, jak i do jego oceny. Oto dlaczego to jest złe.
#Zadaniem tego modelu jest znalezienie wzorców, które przewidują ceny domów, więc zobaczy ten wzór i zawsze będzie przewidywał wysokie ceny domów z zielonymi drzwiami.
#Ponieważ ten wzorzec został wyprowadzony z danych uczących, model będzie wyglądał dokładnie w danych uczących.
#Ale jeśli ten wzorzec nie zachowuje się, gdy model widzi nowe dane, model byłby bardzo niedokładny w praktyce.
#Ponieważ praktyczna wartość modeli wynika z prognozowania nowych danych, mierzymy wydajność na danych, które nie zostały użyte do zbudowania modelu.
# Najprostszym sposobem na to jest wykluczenie niektórych danych z procesu budowania modelu, a następnie użycie ich do przetestowania dokładności modelu na danych, których wcześniej nie widział. 
#Te dane nazywane są danymi walidacyjnymi.

#Podział danych na dane treningowe i walidacyjne, zarówno dla funkcji, jak i celu
#Podział jest oparty na generatorze liczb losowych. Podanie wartości liczbowej do
# argument random_state gwarantuje, że za każdym razem otrzymamy taki sam podział

from sklearn.model_selection import train_test_split

#Podział danych na dane treningowe i walidacyjne, zarówno dla funkcji, jak i celu
#Podział jest oparty na generatorze liczb losowych. Podanie wartości liczbowej do
# argument random_state gwarantuje, że za każdym razem otrzymamy taki sam podział
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Definiowanie modelu
melbourne_model = DecisionTreeRegressor()
# Dopasowanie modelu
melbourne_model.fit(train_X, train_y)

# Przewidywane ceny na danych walidacyjnych
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))


NameError: ignored

In [ ]:
##Kiedy dzielimy domy na wiele liści, mamy również mniej domów na każdym liściu. 
##Liście z bardzo małą liczbą domów będą przewidywać, które są dość zbliżone do rzeczywistych wartości tych domów,
## ale mogą stanowić bardzo niewiarygodne prognozy dla nowych danych (ponieważ każda prognoza opiera się tylko na kilku domach).

#Jest to zjawisko zwane overfittingiem, w którym model prawie idealnie pasuje do danych uczących, ale źle radzi sobie z walidacją i innymi nowymi danymi.
# Z drugiej strony, jeśli sprawimy, że nasze drzewo będzie bardzo płytkie, nie podzieli ono domów na bardzo odrębne grupy.

#Istnieje kilka alternatyw kontrolowania głębokości drzewa, a wiele z nich pozwala, aby niektóre trasy w drzewie miały większą głębokość niż inne trasy.
#Ale argument max_leaf_nodes zapewnia bardzo rozsądny sposób kontrolowania nadmiernego i niedostatecznego dopasowania.
#Im więcej liści pozwolimy modelowi wykonać, tym bardziej przechodzimy od obszaru niedopasowania na powyższym wykresie do obszaru nadmiernego dopasowania.

#Możemy użyć funkcji narzędzia, aby pomóc porównać wyniki MAE z różnych wartości dla max_leaf_nodes:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)


#Dane są ładowane do train_X, val_X, train_y i val_y.
#Można użyć pętli for do porównania dokładności modeli zbudowanych z różnymi wartościami max_leaf_nodes.

# porównanie MAE z różnymi wartościami max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

#Oto wnioski: modele mogą cierpieć na:
#-Overfitting: wychwytywanie fałszywych wzorców, które nie będą się powtarzać w przyszłości, co prowadzi do mniej dokładnych prognoz.
#-Niedopasowanie: brak możliwości uchwycenia odpowiednich wzorców, co ponownie prowadzi do mniej dokładnych prognoz.
#Używamy danych walidacyjnych, które nie są używane w uczeniu modelu, do pomiaru dokładności modelu kandydata. Dzięki temu możemy wypróbować wiele modeli kandydatów i zachować najlepszy.



NameError: ignored

In [ ]:
### LOSOWY LAS

#Losowy las wykorzystuje wiele drzew i dokonuje przewidywań, uśredniając przewidywania każdego drzewa składowego.
#Generalnie ma znacznie lepszą dokładność predykcyjną niż pojedyncze drzewo decyzyjne i działa dobrze z parametrami domyślnymi.
#Jeśli będzie kontynuowane modelowanie, można nauczyć więcej modeli z jeszcze lepszą wydajnością, ale wiele z nich jest wrażliwych na uzyskanie odpowiednich parametrów.


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

##Istnieją parametry, które pozwalają zmienić wydajność losowego lasu, podobnie jak zmieniana była maksymalną głębokość pojedynczego drzewa decyzyjnego.
##Ale jedną z najlepszych cech modeli Random Forest jest to, że generalnie działają one rozsądnie nawet bez tego tuningu.

